**NOTE**: to get this working, you must install the `data-analysis` group dependencies using Poetry:

```shell
poetry install --with=data-analysis
```

You should also run this if you are using git:

```shell
poetry run nbstripout --install
```

This will automatically clear output when committing to git :)

We'll create a basic dataframe from `answers.sqlite3`. Then we'll tease out the JSON columns.

In [ ]:
import pandas as pd
import json
import sqlite3

from sklearn.metrics import cohen_kappa_score

from data_analysis_utils import agreement_as_label

In [ ]:
# Read answers.sqlite3 into a DataFrame
conn = sqlite3.connect("answers.sqlite3")
df = pd.read_sql_query("SELECT * FROM answers", conn)
conn.close()
df

Now time to clean the data — extract it from that JSON column.

Instead of creating a nice schema in `answers.sqlite3`, I decided to defer the job of making nice columns to the data analysis stage. So now we have to parse the answers column as JSON and extract data.


As of 2023-05-02, these are columns:

In [ ]:
df["answers"].apply(json.loads)[0]

In [ ]:
def json_to_columns(df):
    json_column = df["answers"].apply(json.loads)
    return df.assign(
        jargon=json_column.apply(lambda x: x["jargon"]).astype(int),
        sentence_structure=json_column.apply(lambda x: x["sentence_structure"]).astype("category"),
        explanation=json_column.apply(lambda x: x["explanation"]),
        explanation_correctness=json_column.apply(lambda x: x["explanation_correctness"]).astype("category"),
        # I wish I had a better name for this column, but it's basically, "if the explanation is MAYBE correct, WHY is it maybe correct?"
        explanation_maybe=json_column.apply(lambda x: x["explanation_maybe"]).astype("category"),
        fix=json_column.apply(lambda x: x["fix"]).astype("category"),
        fix_correctness=json_column.apply(lambda x: x["fix_correctness"]).astype("category"),
        additional_errors=json_column.apply(lambda x: x["additional_errors"]).astype("category"),
        notes=json_column.apply(lambda x: x["notes"]),
        length=json_column.apply(lambda x: x["length"]).astype(int),
    )


full_df = df.pipe(json_to_columns)\
    .drop(columns=["answers"])
full_df

In [ ]:
full_df.info()

In [ ]:
two_ratings = full_df.groupby(["srcml_path", "version"]).filter(lambda x: len(x["rater"].unique()) == 2)
two_ratings

In [ ]:
two_ratings["explanation"].value_counts()

In [ ]:
# Get the explanation ratings from prajish and eddie:
eddie_ratings = two_ratings[two_ratings["rater"] == "eddie"][["srcml_path", "version", "variant", "explanation"]]
prajish_ratings = two_ratings[two_ratings["rater"] == "prajish"][["srcml_path", "version", "variant", "explanation"]]

# Join the two ratings together
ratings = eddie_ratings.merge(prajish_ratings, on=["srcml_path", "version", "variant"], suffixes=["_eddie", "_prajish"])

In [ ]:
# Get the ratings for the two raters
eddie_ratings = two_ratings[two_ratings["rater"] == "eddie"]
prajish_ratings = two_ratings[two_ratings["rater"] == "prajish"]
brett_ratings = two_ratings[two_ratings["rater"] == "brett"]
assert len(eddie_ratings) == len(prajish_ratings) == len(brett_ratings)
assert len(eddie_ratings) == 40

# Join eddie, brett, prajish ratings togtether
#ratings = eddie_ratings.merge(prajish_ratings, on=["srcml_path", "version", "variant"], suffixes=["_eddie", "_prajish"])\
#    .merge(brett_ratings, on=["srcml_path", "version", "variant"], suffixes=["", "_brett"])

ratings = eddie_ratings.merge(prajish_ratings, on=["srcml_path", "version", "variant"], suffixes=["_eddie", "_prajish"])

In [ ]:
brett_ratings.merge(eddie_ratings, on=["srcml_path", "version", "variant"], suffixes=["_brett", "_eddie"]).sample(5)

In [ ]:
# TODO: perhaps Scott's pi is a better measure of agreement for this data:
# See: https://stats.stackexchange.com/a/525640

kappa = cohen_kappa_score(ratings["explanation_eddie"], ratings["explanation_prajish"])
kappa, agreement_as_label(kappa)

In [ ]:
kappa = cohen_kappa_score(ratings["fix_eddie"], ratings["fix_prajish"])
kappa, agreement_as_label(kappa)